In [1]:
import numpy as np
import pandas as pd
import torch

# Custom modules
from approximators import MLP, QValueApproximator
from states import EnvironmentState
from agents import LearningAgent
from data import DataGenerator

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

In [2]:
n_levels = 4
horizon = 50
action_size = 10

data = DataGenerator.generator("../data/AAPL.parquet", levels=n_levels)
state = EnvironmentState(data, n_levels, action_size)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

model = MLP(10, 1, [32, 32], torch.nn.ReLU, 0.1, device).to(device)
q_approximator = QValueApproximator(model, 1e-3)

Using cuda device


In [4]:
agent = LearningAgent(
    action_size=action_size,
    q_value_approximator=q_approximator,
    epsilon=0.5,
    epsilon_decay=0.99,
    min_epsilon=0.01,
    gamma=0.99,
)

env = EnvironmentState(data, n_levels, horizon)

In [5]:
n_episodes = 10

agent.train(env, n_episodes)

Time 0
Action 6
Orders
[(586.5299999999999, 100, 'ask'), (585.93, 100, 'bid')]

Order 0
Order 1
Order completed
Next state [ 5.70000000e-01  1.50000000e-02 -6.94915254e-01 -8.20000000e+01
  8.29727627e-06  6.25000000e+01  1.00000000e+02  3.00000000e+00
  1.00000000e+00]
State [ 6.0000002e-01  0.0000000e+00 -6.9491524e-01 -8.2000000e+01
  3.5999046e-06  4.0000000e+01  0.0000000e+00  3.0000000e+00
  1.0000000e+00  6.0000000e+00]
tensor([ 6.0000e-01,  0.0000e+00, -6.9492e-01, -8.2000e+01,  3.5999e-06,
         4.0000e+01,  0.0000e+00,  3.0000e+00,  1.0000e+00,  6.0000e+00],
       device='cuda:0')
tensor([ 5.7000e-01,  1.5000e-02, -6.9492e-01, -8.2000e+01,  8.2973e-06,
         6.2500e+01,  1.0000e+02,  3.0000e+00,  1.0000e+00,  0.0000e+00],
       device='cuda:0')
tensor([ 5.7000e-01,  1.5000e-02, -6.9492e-01, -8.2000e+01,  8.2973e-06,
         6.2500e+01,  1.0000e+02,  3.0000e+00,  1.0000e+00,  1.0000e+00],
       device='cuda:0')
tensor([ 5.7000e-01,  1.5000e-02, -6.9492e-01, -8.2000e+

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn